<a href="https://colab.research.google.com/github/zzeeddd95/capstone-project/blob/master/SVR%2Blinearregapple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas_datareader as web


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [31]:
#Retrieve stock data
apple = web.DataReader('AAPL', data_source='yahoo', start='2010-05-27', end='2020-07-06')
print(apple.head())

                 High        Low       Open      Close       Volume  Adj Close
Date                                                                          
2010-05-27  36.270000  35.587143  35.799999  36.192856  166570600.0  31.332062
2010-05-28  37.057144  36.192856  37.055714  36.697144  203903700.0  31.768623
2010-06-01  37.991428  36.994286  37.098572  37.261429  219118200.0  32.257130
2010-06-02  37.828571  37.189999  37.791428  37.707142  172137000.0  32.642975
2010-06-03  37.935715  37.201427  37.882858  37.588570  162526700.0  32.540325


In [32]:
#Filter out data to only obtain close price (independant variable)
apple = apple[['Close']]
apple

,Close
Date,
2010-05-27,36.192856
2010-05-28,36.697144
2010-06-01,37.261429
2010-06-02,37.707142
2010-06-03,37.588570
...,...
2020-06-29,361.779999
2020-06-30,364.799988
2020-07-01,364.109985


In [33]:
#Create a variable to allow for predicting 'n days' into the future
future_out =30

In [34]:
#Create the dependant variable which will be shifted 'n' units up
apple['Prediction'] = apple[['Close']].shift(-future_out)
apple.tail()

,Close,Prediction
Date,,
2020-06-29,361.779999,NaN
2020-06-30,364.799988,NaN
2020-07-01,364.109985,NaN
2020-07-02,364.109985,NaN
2020-07-06,373.850006,NaN


In [35]:
#create independant data set by converting the dataframe to a numpy array
appledf = np.array(apple.drop(['Prediction'],1))
#Renove the last 'n' rows
appledfx = appledf[:-future_out] #therefore for every column, all of the rows will displayed - future out
appledfx

array([[ 36.19285583],
       [ 36.69714355],
       [ 37.26142883],
       ...,
       [313.14001465],
       [319.23001099],
       [316.8500061 ]])

In [36]:
#creat dependant dataset (prediction), all values including any Nan's
appledfy = np.array(apple['Prediction'])
#Filter this data by obtaining all values except the last 'n' rows
appledfy = appledfy[:-future_out]
appledfy

array([ 36.75571442,  35.97142792,  36.10428619, ..., 364.10998535,
       364.10998535, 373.8500061 ])

In [37]:
#split data into 80% training and 20%testing
appledfx_train , appledfx_test, appledfy_train, appledfy_test = train_test_split(appledfx, appledfy, test_size = 0.2)

In [38]:
#Create and train the SVR model
SVRapple = SVR(kernel='rbf', C=1e3,gamma=0.1)
SVRapple.fit(appledfx_train, appledfy_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [39]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
SVMconfidence = SVRapple.score(appledfx_test, appledfy_test)
print("SVM confidence", SVMconfidence)

SVM confidence 0.9460487658466542


In [40]:
#Create and train the linear regression model
linear = LinearRegression()
#train the model
linear.fit(appledfx_train, appledfy_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [41]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
linearconfidence = linear.score(appledfx_test, appledfy_test)
print("Linear regression confidence", linearconfidence)

Linear regression confidence 0.9461680839501049


In [42]:
#set apple_forecast = to the last 30 rows of original dataset from close price column
apple_forecast = np.array(apple.drop(['Prediction'],1))[-future_out:]
print(apple_forecast)


[[318.89001465]
 [316.73001099]
 [318.10998535]
 [318.25      ]
 [317.94000244]
 [321.8500061 ]
 [323.33999634]
 [325.11999512]
 [322.32000732]
 [331.5       ]
 [333.45999146]
 [343.98999023]
 [352.83999634]
 [335.8999939 ]
 [338.79998779]
 [342.98999023]
 [352.07998657]
 [351.58999634]
 [351.73001099]
 [349.72000122]
 [358.86999512]
 [366.52999878]
 [360.05999756]
 [364.83999634]
 [353.63000488]
 [361.77999878]
 [364.79998779]
 [364.10998535]
 [364.10998535]
 [373.8500061 ]]


In [43]:
#Print the SVR predicted values for the upcoming 'n' days
SVRprediction = SVRapple.predict(apple_forecast)
print(SVRprediction)

[256.00399978 299.50208824 270.32192706 267.46513295 273.8905849
 244.10980849 240.20376177 257.15270671 242.46852623 194.96039736
 189.97419865 189.31037569 189.31037569 189.32749684 189.31042375
 189.31037569 189.31037569 189.31037569 189.31037569 189.31037569
 189.31037569 189.31037569 189.31037569 189.31037569 189.31037569
 189.31037569 189.31037569 189.31037569 189.31037569 189.31037569]


In [44]:
#Print the linear regression predicted values for the upcoming 'n' days
linearprediction = linear.predict(apple_forecast)
print(linearprediction)

[328.11134197 325.89066535 327.30940221 327.45334977 327.1346446
 331.15447746 332.68632045 334.51631812 331.63768068 341.07553276
 343.09057897 353.91635787 363.01495429 345.59911933 348.58057198
 352.88826858 362.2335964  361.72984269 361.87379025 359.80732075
 369.21433141 377.08949909 370.43776017 375.35202569 363.82715361
 372.20607499 375.31089333 374.60150922 374.60150922 384.61512017]


In [47]:
#Retrieve stock data
applestock = web.DataReader('AAPL', data_source='yahoo', start='2020-05-28', end='2020-07-05')
applestock = applestock[['Close']]
applestock.head()

,Close
Date,
2020-05-28,318.250000
2020-05-29,317.940002
2020-06-01,321.850006
2020-06-02,323.339996
2020-06-03,325.119995
